# QA Fetch Smoke v1\n\nGoal: verify resolver + data connectivity and print `head/columns/dtypes/len`.\n\nBefore running, make sure your DB env vars are loaded in this notebook kernel.

In [ ]:
import sys\nfrom pathlib import Path\n\nrepo = Path.cwd()\nif str(repo / 'src') not in sys.path:\n    sys.path.insert(0, str(repo / 'src'))\n\nprint('repo:', repo)

In [ ]:
from quant_eam.qa_fetch.resolver import qa_fetch_registry_payload, resolve_fetch, fetch_market_data\n\npayload = qa_fetch_registry_payload()\nprint('functions:', len(payload['functions']))\nprint('resolver_entries:', len(payload['resolver_entries']))\n\nfor row in payload['resolver_entries'][:12]:\n    adv = row['adjustment']['adv'] or {}\n    print(row['asset'], row['freq'], row.get('venue'), 'raw=', row['raw']['public_name'], 'adv=', adv.get('public_name'))

In [ ]:
import pandas as pd\n\ndef show_result(tag, obj):\n    print(f'[{tag}] type=', type(obj))\n    if obj is None:\n        print('  -> None')\n        return\n    if isinstance(obj, pd.DataFrame):\n        print('  len=', len(obj))\n        print('  columns=', list(obj.columns))\n        print('  dtypes=', {k: str(v) for k, v in obj.dtypes.items()})\n        display(obj.head())\n        return\n    data = getattr(obj, 'data', None)\n    if isinstance(data, pd.DataFrame):\n        print('  len=', len(data))\n        print('  columns=', list(data.columns))\n        print('  dtypes=', {k: str(v) for k, v in data.dtypes.items()})\n        display(data.head())\n        return\n    if hasattr(obj, 'head'):\n        try:\n            print(obj.head())\n        except Exception as e:  # noqa: BLE001\n            print('  head() failed:', e)

In [ ]:
cases = [\n    {\n        'tag': 'stock_day_raw',\n        'asset': 'stock',\n        'freq': 'day',\n        'symbols': ['000001'],\n        'start': '2024-01-02',\n        'end': '2024-01-12',\n        'adjust': 'raw',\n    },\n    {\n        'tag': 'stock_day_qfq',\n        'asset': 'stock',\n        'freq': 'day',\n        'symbols': ['000001'],\n        'start': '2024-01-02',\n        'end': '2024-01-12',\n        'adjust': 'qfq',\n    },\n    {\n        'tag': 'future_day_raw',\n        'asset': 'future',\n        'freq': 'day',\n        'symbols': ['rb2401'],\n        'start': '2024-01-02',\n        'end': '2024-01-12',\n        'adjust': 'raw',\n    },\n    {\n        'tag': 'bond_day_raw',\n        'asset': 'bond',\n        'freq': 'day',\n        'symbols': 'all',\n        'start': '2024-01-02',\n        'end': '2024-01-12',\n        'adjust': 'raw',\n    },\n    {\n        'tag': 'bond_day_cfets_raw',\n        'asset': 'bond',\n        'freq': 'day',\n        'venue': 'cfets',\n        'symbols': 'all',\n        'start': '2024-01-02',\n        'end': '2024-01-12',\n        'adjust': 'raw',\n    },\n]\n\nfor case in cases:\n    tag = case['tag']\n    print('\n===', tag, '===')\n    try:\n        r = resolve_fetch(\n            asset=case['asset'],\n            freq=case['freq'],\n            venue=case.get('venue'),\n            adjust=case.get('adjust', 'raw'),\n        )\n        print('resolved:', r)\n        out = fetch_market_data(\n            asset=case['asset'],\n            freq=case['freq'],\n            venue=case.get('venue'),\n            symbols=case['symbols'],\n            start=case['start'],\n            end=case['end'],\n            adjust=case.get('adjust', 'raw'),\n            format='pd',\n        )\n        show_result(tag, out)\n    except Exception as e:  # noqa: BLE001\n        print('ERROR:', type(e).__name__, str(e))